<center>Progetto realizzato da Elena Curti (matr. 185431)

# Ricette
</center>

## Introduzione


## Requisiti
Questo progetto è stato realizzato con Python 3.11 e Neo4j. E' necessaria l'installazione dei seguenti pacchetti:

In [1]:
# %pip install py2neo neo4j-driver pandas

## Operazioni iniziali
Per il corretto funzionamento del progetto proposto, è necessario:
- Creare un nuovo progetto Neo4j 
- Creare all'interno del progetto un DMBS (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0)
- Aggiungere il plugin APOC.
- Premere su Start.
<!-- 
Da Neo4j creare un nuovo progetto. Scegliere una tra le seguenti alternative:
- Dal progetto appena creato, aggiungere al progetto il file [neo4j.dump](data/neo4j.dump). Creare poi un nuovo DBMS dal file dump (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0). Aggiungere il plugin APOC. Premere su Start.
- Dal progetto appena creato, aggiungere un DBMS locale (scegliendo come nome, password e versione i valori indicati al punto sopra). Aggiungere il plugin APOC. Premere su Start e poi su Open. Eseguire tutti i comandi riportati nel file [import_commands.txt](data/import_commands.txt).
<br>-->
Eseguire poi le celle di codice, nell'ordine proposto 

## Connessione e autenticazione
Per effettuare la connessione al database, eseguire il seguente script:

In [2]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "pass"))

print("Connessione al database eseguita correttamente!")

Connessione al database eseguita correttamente!


# Schema e dati originali
Lo use case originale è stato trovato sul sito [Recipes Neo4j](https://neo4j.com/graphgists/dd3dedcf-c377-4575-84f4-4d0d30b2a4c5/). Ha la seguente struttura iniziale:
```
    call db.schema.visualization()
```

![schema](img/db_schema2.png)

I nodi Ingredient, Author, Collection, DietType, Keyword hanno un solo campo "name" che ne rappresenta il nome. <br>
Il nodo Recipe ha le seguenti proprietà:
<ul>
    <li>id: campo che identifica univocamente la ricetta </li>
    <li>cookingTime: tempo di cottura </li>
    <li>description: descrizione della ricetta </li>
    <li>name: nome della ricetta </li>
    <li>preparationTime: tempo di preparazione </li>
    <li>skillLevel: difficoltà </li>
</ul>
Gli archi non hanno proprietà. <br><br>

Per importare i dati e aggiungere gli indici ho usato il seguente script. <br>
Lo usecase originale ha previsto i seguenti indici per migliorare la performance delle cypher query:
    <ul>
        <li><i>id</i> su Recipe</li>
        <li><i>name</i> su Ingredient</li>
        <li><i>name</i> su Keyword</li>
        <li><i>name</i> su DietType</li>
        <li><i>name</i> su Author</li>
        <li><i>name</i> su Collection</li>
    </ul>
Il comando ```CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id)``` è equivalente a ```CREATE INDEX IF NOT EXISTS ON :Recipe(id)```. Il primo, però, è compatibile con la versione 5.2.0 di Neo4j.

In [2]:
print("Inserimento dei dati in corso... ")

# Ricette
graph.run(""" 
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.title AS title,
       value.page.article.description AS description,
       value.page.recipe.cooking_time AS cookingTime,
       value.page.recipe.prep_time AS preparationTime,
       value.page.recipe.skill_level AS skillLevel
MERGE (r:Recipe {id: id})
SET r.cookingTime = cookingTime,
    r.preparationTime = preparationTime,
    r.name = title,
    r.description = description,
    r.skillLevel = skillLevel;
""")
graph.run(""" CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id);        // CREATE INDEX IF NOT EXISTS ON :Recipe(id)  """)

# Autori
graph.run("""
//import authors and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.article.author AS author
MERGE (a:Author {name: author})
WITH a,id
MATCH (r:Recipe {id:id})
MERGE (a)-[:WROTE]->(r);
""")
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Author) ON (n.name);""" )

# Ingredienti
graph.run("""//import ingredients and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.ingredients AS ingredients
MATCH (r:Recipe {id:id})
FOREACH (ingredient IN ingredients |
  MERGE (i:Ingredient {name: ingredient})
  MERGE (r)-[:CONTAINS_INGREDIENT]->(i)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Ingredient) ON (n.name);""" )

# DietType
graph.run("""//import dietTypes and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.diet_types AS dietTypes
MATCH (r:Recipe {id:id})
FOREACH (dietType IN dietTypes |
  MERGE (d:DietType {name: dietType})
  MERGE (r)-[:DIET_TYPE]->(d)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:DietType) ON (n.name);""" )

# Collections
graph.run("""//import collections and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.collections AS collections
MATCH (r:Recipe {id:id})
FOREACH (collection IN collections |
  MERGE (c:Collection {name: collection})
  MERGE (r)-[:COLLECTION]->(c)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Collection) ON (n.name);""" )

# Keyword
cq = """//import keywords and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.keywords AS keywords
MATCH (r:Recipe {id:id})
FOREACH (keyword IN keywords |
  MERGE (k:Keyword {name: keyword})
  MERGE (r)-[:KEYWORD]->(k)
);"""
# graph.run(cq)
cq = """CREATE INDEX IF NOT EXISTS FOR (n:Keyword) ON (n.name);""" 
# graph.run(cq)


print("Tutti i dati sono stati importati correttamente!")

Inserimento dei dati in corso... 
Tutti i dati sono stati importati correttamente!


## Modifiche effettuate
Per arricchire il progetto ho deciso di aggiungere dei dati presi da Kaggle e di eseguire alcune modifiche allo schema originale. 
<br>
<mark>TODOaggiunti altri campi dal file originale  <br>



In [3]:
# Definisco un'eccezione usata nel seguito
class MiaStopExecution(Exception):
    """Il raise di questa classe provoca l'interruzione dell'esecizione della cella, senza interrompere il kernel"""
    def _render_traceback_(self):
        pass

# Prima di effettuare le modifiche, copio i file necessari nella cartella "import" del DBMS, mediante il seguente script:
import os 
import shutil
DATA_FOLDER ="data"+os.path.sep

cq ="""CALL dbms.listConfig() YIELD name, value
    WHERE name="server.directories.import"
    RETURN value"""
DIRECTORY_IMPORT = graph.run(cq).evaluate()
print("Cartella import del DMBS: " + DIRECTORY_IMPORT)

def copy_in_import_dir(file_name):
    source = DATA_FOLDER + file_name
    destination = DIRECTORY_IMPORT + os.path.sep + file_name
    shutil.copy(source, destination)

copy_in_import_dir("FoodData.csv") 
copy_in_import_dir("gz_recipe.csv") 


Cartella import del DMBS: D:\Elena\_Elena\Shared\Universita\Magistrale\Big_data\Anno_22_23\TO_DEL_QUANDO_ESAME_DATO\Neo4J\relate-data\dbmss\dbms-b3e0a01c-d1be-4257-8d27-97054bc0e187\import


### 1. Rimozione di Keyword
Ho deciso di rimuovere il nodo Keyword. Ho quindi evitato di eseguire le ultime due cypher query nello script precedente. Alternativamente, avrei potuto anche usare il seguente comando: 

In [4]:
graph.run("MATCH (n:Keyword) DETACH DELETE n")

(No data)

### 2. Aggiunta di altre ricette
Per arricchire il dataset, ho deciso di inserire altre ricette al database. Ho quindi scaricato il file [gz_recipe.csv](data/gz_recipe.csv) da https://www.kaggle.com/datasets/edoardoscarpaci/italian-food-recipes, contenente delle ricette estratte dal sito [GialloZafferano](https://www.giallozafferano.it/). <br>
Il file scaricato contiene i seguenti campi:
- id: id che identifica univocamente la ricetta (intero da 0 a 5938).
- Nome: nome del piatto
- Categoria: categoria della ricetta (Primi, Secondi, ...)
- Link: link della ricetta
- Persone/Pezzi: numero di persone o pezzi della ricetta
- Ingredienti: lista di ingredienti e relative quantità. Ad esempio: [['Mascarpone', '750g'], ['Uova', '260g']]
- Steps: contenuto della ricetta

Per i inserire questi dati ho deciso di:
- Aggiungere un nodo Fonte con una sola property name (univoco) per indicare la fonte della ricetta ("GialloZafferano" o "BBC GoodFood"). 
- Unire Fonte a Recipe 
- Aggiungere a Recipe le proprietà 
    - "persone_pezzi" per indicare il numero di persone o pezzi
    - "link" contenente il link <mark> TODO </mark>
- Aggiungere a CONTAINS_INGREDIENTS una proprietà "quantita", contenente la quantità di un ingrediente in una ricetta

Come precedentemente spiegato, il campo id di Recipe è univoco nelle ricette già inserite. Aggiungendo i dati da un altra sorgente, si potrebbero avere dei record con lo stesso id. Prima di inserire le nuove ricette, quindi, controllo che gli id delle nuove ricette siano tutti diversi da quelli delle ricette già inserite. In questo caso, per fortuna, non ci sono conflitti sugli id e si può procedere con gli inserimenti. 

In [5]:
# Aggiungo le ricette di GialloZafferano. 

# Controllo che non ci siano già ricette con gli stessi id
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MATCH (r:Recipe {id:value["id"]})
WHERE NOT EXISTS ( (r)-[:FONTE]-(:Fonte {name:"GialloZafferano"}) )
return count(r)
"""
if graph.run(cq).evaluate() != 0:
    print("Le ricette di BBC e quelle di GialloZafferano hanno degli id in comune!")
    raise(MiaStopExecution)

# Creo il nodo Fonte con: name univoco, indice su name. Creo poi le due fonti
cq = ["CREATE CONSTRAINT nome_fonte_univoco IF NOT EXISTS FOR (f:Fonte) REQUIRE f.name IS UNIQUE"]
cq += ["MERGE (n:Fonte {name:'GialloZafferano'})"]
cq += ["MERGE (n:Fonte {name:'BBC GoodFood'})"]
[graph.run(i) for i in cq]


# Inserisco le ricette
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MATCH (f:Fonte {name:"GialloZafferano"})
MERGE (r:Recipe {id: value["id"]})<-[:FONTE]-(f)
SET r.name = value["Nome"],
    r.description = value["Steps"],
    r.persone_pezzi = value["Persone/Pezzi"]
"""
graph.run(cq)

cq="""
MATCH (gz:Fonte {name:"GialloZafferano"}),(bbc:Fonte {name:"BBC GoodFood"}), (r:Recipe) 
WHERE NOT EXISTS ( (r)<-[:FONTE]-(gz) ) 
MERGE (r)<-[:FONTE]-(bbc)
"""
graph.run(cq)

# Inserisco gli ingredienti
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
WITH value["id"] AS id, replace(value["Ingredienti"], '"', "") AS ris1
WITH id, replace(ris1, "[[", "") AS ris2
WITH id,replace(ris2, "]]", "") AS ris3
WITH id,replace(ris3, "'", "") AS ris4
WITH id,split(ris4, "], [") AS ingrs_list
MATCH (r:Recipe {id:id})
WHERE ingrs_list[0]<>"[]"   // Alcune ricette non hanno ingredienti nè descrizione (es. Churros red velvet)
FOREACH (ingr_quantita_string IN ingrs_list |
    MERGE (i:Ingredient {name: split(ingr_quantita_string, ", ")[0]})
    MERGE (r)-[:CONTAINS_INGREDIENT {quantita: split(ingr_quantita_string, ", ")[1]}]->(i)
)
""" 
graph.run(cq)


# Creo le categorie 
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MATCH (r:Recipe {id:value["id"]})
WHERE value["Categoria"] IS NOT NULL
MERGE (c:Collection {name: value["Categoria"]})
MERGE (r)-[:COLLECTION]->(c)
"""
graph.run(cq)



(No data)

### 3. Inserimento degli allergeni
Ho aggiunto un campo "is_allergene" al nodo Ingredient. Ho scaricato da Kaggle due file:
- [allergens.json](data/allergens.json) (https://www.kaggle.com/datasets/elenacivati/allergensjson): contenente un elenco non molto ampio di allergeni in italiano, inglese e altre lingue.
- [FoodData.csv](data/FoodData.csv) (https://www.kaggle.com/datasets/boltcutters/food-allergens-and-allergies): contenente un elenco più ampio ma solo in inglese

Ho quindi impostato gli ingredienti contenuti nei file con is_allergene=True. Ho invece settato is_allergene=False per i restanti ingredienti. 

In [101]:
# Prendo dal file allergens.json i nomi degli ingredienti in inglese e italiano. 
# Converto poi in csv, in modo da semplificare la lettura del file successivamente
import pandas as pd
df_allergens = pd.read_json(DATA_FOLDER + "allergens.json").transpose()
df_allergens["en"]=df_allergens.index
df_allergens["en"]=df_allergens["en"].apply(lambda x: x[3:])
df_allergens["it"]=df_allergens["name"].apply(lambda x: x["it"])
df_allergens=df_allergens[["en", "it"]]
df_allergens.to_csv(DATA_FOLDER + "allergens.csv", index=False)
copy_in_import_dir("allergens.csv")

# Setto gli allergeni
cq="""
LOAD CSV WITH HEADERS FROM 'file:///FoodData.csv' AS value
LOAD CSV WITH HEADERS FROM 'file:///allergens.csv' AS value2
MATCH (i:Ingredient)
WHERE 
    toLower(i.name) CONTAINS toLower(value["Food"])  OR 
    toLower(i.name) CONTAINS toLower(value2["en"])  OR 
    toLower(i.name) CONTAINS toLower(value2["it"])  
SET i.is_allergene=True;"""
graph.run(cq)

# Setto i restanti ingredienti come non allergeni
cq="""
MATCH (i:Ingredient)
WHERE i.is_allergene IS NULL
SET i.is_allergene=False; """
graph.run(cq)


(No data)

### 4. Nuovo indice
Per velocizzare in futuro la ricerca delle ricette in base al nome, ho deciso di inserire un nuovo indice sulla proprietà name di Recipe.

In [8]:
graph.run(" CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.name);")

(No data)

### Schema finale
Dopo le modifiche, lo schema finale è il seguente:

![schema](img/db_schema_finale.png)

In [89]:
def print_first_n(cq, n=1000):
    ris = graph.run(cq)
    ris.sample_size = n
    return ris

print_first_n("CALL db.schema.nodeTypeProperties")

nodeType,nodeLabels,propertyName,propertyTypes,mandatory
:`Author`,['Author'],name,['String'],true
:`Ingredient`,['Ingredient'],name,['String'],true
:`Ingredient`,['Ingredient'],is_allergene,['Boolean'],true
:`DietType`,['DietType'],name,['String'],true
:`Collection`,['Collection'],name,['String'],true
:`Recipe`,['Recipe'],id,['String'],true
:`Recipe`,['Recipe'],cookingTime,['Long'],false
:`Recipe`,['Recipe'],preparationTime,['Long'],false
:`Recipe`,['Recipe'],name,['String'],true
:`Recipe`,['Recipe'],description,['String'],false


In [10]:
print_first_n("CALL db.schema.relTypeProperties")

relType,propertyName,propertyTypes,mandatory
:`WROTE`,null,null,false
:`CONTAINS_INGREDIENT`,quantita,['String'],false
:`DIET_TYPE`,null,null,false
:`COLLECTION`,null,null,false


## Interrogazioni
[ok] - Cerca le ricette con due ingredienti scelti
[ok] - Cerca una ricetta in base al nome 
[ok] - Ricetta più veloce da preparare
[ok] - Cerca lo chef che ha scritto più ricette
[ok] - Stampa le ricette senza allergeni
[ok] - Partendo da una ricetta, mostra i suggerimenti (collection, diet type, autore, skilLevel, fonte)
- Rimettere Keyword e "The Top 5 tags co-occurring with keyword ..., with respective frequency count"



### Quali ricette contengono due ingredienti (scelti dall'utente)?

In [90]:
# Cerco le ricette con due ingredienti
print("************ Stampa una ricetta con due ingredienti ************ ")

# Stampo qualche suggerimento
print("Qualche suggerimento per la scelta degli ingredienti: ")
cq = """ 
MATCH (i:Ingredient)
WITH i, rand() as num
RETURN i.name AS ingrediente, i.is_allergene AS is_allergene
ORDER BY num
LIMIT 5
"""
display(print_first_n(cq))

# Chiedo il primo ingrediente e stampo gli ingredienti più usati con esso
ingrediente1=""
while ingrediente1=="":
    ingrediente1=input("Inserisci primo ingrediente: ")
# ingrediente1="Confettura di prugne"

print("Ecco 5 ingredienti usati spesso con " +ingrediente1)
cq = """ 
MATCH (i_scelto:Ingredient)<-[:CONTAINS_INGREDIENT]-(:Recipe)-[:CONTAINS_INGREDIENT]->(i2:Ingredient)
WHERE toLower(i_scelto.name) CONTAINS toLower('"""+ingrediente1+"""')
WITH i2,  COUNT(i2) AS num
RETURN i2.name AS ingrediente_raccomandato, i2.is_allergene AS is_allergene
ORDER BY num DESC
LIMIT 5
"""
display(print_first_n(cq))


# Chiedo il secondo ingrediente
ingrediente2=""
while ingrediente2=="": 
    ingrediente2=input("Inserisci secondo ingrediente: ")
# ingrediente2="Uova"

# Cerco i risultati...
cq = """ 
MATCH (i1:Ingredient)<-[:CONTAINS_INGREDIENT]-(r:Recipe)-[:CONTAINS_INGREDIENT]->(i2:Ingredient)
WHERE 
    toLower(i1.name) CONTAINS toLower('"""+ingrediente1+"""') AND 
    toLower(i2.name) CONTAINS toLower('"""+ingrediente2+"""') 
RETURN r
"""
recipe_list = graph.run(cq)

# ... e li stampo
print("\nRicette trovate con gli ingredienti",ingrediente1, ", " ,ingrediente2,"\b: ")
ricetta_trovata = False
for r in recipe_list:
    ricetta_trovata = True
    node_recipe = r[r.keys()[0]]
    print("\t", node_recipe.get("name", "\b"))
if not ricetta_trovata:
    print("------ Nessuna ricetta! ------")
    

************ Stampa una ricetta con due ingredienti ************ 
Qualche suggerimento per la scelta degli ingredienti: 


ingrediente,is_allergene
mixed spice,false
Risotto allo zafferano,false
Mele Cotogne,false
cream cracker,true
Barolo,false


Ecco 5 ingredienti usati spesso con Confettura di Prugne


ingrediente_raccomandato,is_allergene
Uova,true
Farina 00,false
Zucchero,false
Sale fino,false
Latte intero,true



Ricette trovate con gli ingredienti Confettura di Prugne ,  Zucchero: 
	 Girelle alla marmellata
	 Treccine di sfoglia
	 Crostata alla crema e prugne
	 Crostata alla crema e prugne


### Stampa i dettagli di una ricetta

In [12]:
# Chiedo il nome della ricetta
nome_ricetta=""
while nome_ricetta=="": 
    nome_ricetta=input("Inserisci nome della ricetta: ")
print("Ricetta scelta:", nome_ricetta)

# Cerco la ricetta e tutti i relativi dati
cq = """MATCH (r:Recipe) 
WHERE toLower(r.name) = toLower('"""+nome_ricetta+"""')
OPTIONAL MATCH (a:Author)-[:WROTE]->(r)
OPTIONAL MATCH (c:Collection)<-[:COLLECTION]-(r)
OPTIONAL MATCH (d:DietType)<-[:DIET_TYPE]-(r)
OPTIONAL MATCH (i:Ingredient)<-[r_i:CONTAINS_INGREDIENT]-(r)
RETURN r AS ricetta, COLLECT(DISTINCT c) AS categorie, COLLECT(DISTINCT i) AS ingredienti, COLLECT(DISTINCT d) AS tipi_dieta, COLLECT(DISTINCT r_i) AS quantita_ingredienti
"""
list_recipes = graph.run(cq)

# Stampo i risultati
ricetta_trovata=False
for res in list_recipes:
    ricetta_trovata = True

    tmp = dict(res)
    print("---------------")

    if tmp.__contains__("ricetta"):
        for k,v in tmp["ricetta"].items():
            print(k, "\b:",v)
    
    if tmp.__contains__("categorie"):
        categorie = tmp["categorie"]
        print("Categorie:", [v for c in categorie for _,v in c.items()])
    
    if tmp.__contains__("ingredienti"):
        ingredienti =  [c["name"] + (" (Allergene!)" if c["is_allergene"] else "") for c in tmp["ingredienti"]]
    
    if tmp.__contains__("quantita_ingredienti"):
        quantita_ingredienti = [dict(q_i).get("quantita", "") for q_i in tmp["quantita_ingredienti"]]
        tmp2 = []
        for i in range(len(ingredienti)):
            tmp2.append( ingredienti[i] + " " + quantita_ingredienti[i])
        ingredienti = tmp2
    
    print("Ingredienti:", ingredienti)

if not ricetta_trovata:
    print("--- Nessuna ricetta trovata --- ")


Ricetta scelta: chocoloate chia pudding
--- Nessuna ricetta trovata --- 


### Quali sono le ricette più veloci da preparare?

In [88]:
print("Scegli un'opzione. Ricetta piu' veloce in base a: \n\t1. Tempo di preparazione\n\t2. Tempo di cottura\n\t3. Tempo totale")
opzione=""
while opzione=="":
    opzione=input(">> ")
    opzione = "tempo_preparazione" if opzione=="1" else ("tempo_cottura" if opzione=="2" else ("tempo_totale" if opzione=="3" else ""))
print("Opzione scelta: " + opzione)

cq = """MATCH (r:Recipe)
WHERE r.preparationTime<>0
RETURN r.name AS ricetta, r.cookingTime AS tempo_cottura, r.preparationTime as tempo_preparazione, r.cookingTime+r.preparationTime AS tempo_totale
ORDER BY """ + opzione + " LIMIT 3"
graph.run(cq)


Scegli un'opzione. Ricetta piu' veloce in base a: 
	1. Tempo di preparazione
	2. Tempo di cottura
	3. Tempo totale
Opzione scelta: tempo_preparazione


ricetta,tempo_cottura,tempo_preparazione,tempo_totale
Lighter Cornish pasties,3000,60,3060
Christmas spice latte,180,60,240
Creamy yogurt porridge,180,60,240


### Chi ha scritto più ricette?

In [91]:
cq = """MATCH (a:Author)-[:WROTE]->(r:Recipe)
WITH a, COUNT(r) AS num_ricette, COLLECT(r.name)[..5] AS alcune_ricette // Seleziono solo le prime cinque ricette
RETURN a.name, num_ricette, alcune_ricette
ORDER BY num_ricette DESC
LIMIT 5
"""
display(print_first_n(cq))

a.name,num_ricette,alcune_ricette
Good Food,3441,"['Zesty lentil & haddock pilaf', 'Fruity figgy flapjacks', 'Beef & mozzarella meatballs', 'Stuffed summer tomatoes', 'Lamb kebabs & Greek salad']"
Sara Buenfeld,664,"['Very simple Margherita pizza', 'Sweet & spicy apricot chicken', 'Healthy tuna lettuce wraps', 'Prawn & fennel bisque', 'Blueberry soured cream cake with cheesecake frosting']"
Barney Desmazery,646,"['Christmas poussin', 'Stuffed pork medallions', 'Cheese & Marmite sausage rolls', 'Violet creams', 'Next level apple crumble']"
Cassie Best,638,"['Pear, nut & blackberry Bircher', 'Chicken with mustard lentils', 'Pea & bacon pasties', 'Pepper, pesto & sweetcorn calzones', 'Mini Christmas cake']"
Sarah Cook,515,"['Leftover veg & orange cake', 'Free-form asparagus & potato tart', 'Baked camembert pie for sharing', 'Chicken, apricot & pork pie', 'Malted walnut seed loaf']"


### Quali ricette non contengono allergeni?

In [92]:
cq = """
MATCH (r:Recipe)-[:CONTAINS_INGREDIENT]->(i:Ingredient)
WITH r, COLLECT(i.is_allergene) AS are_allergeni, COLLECT(i.name) AS ingredienti
WHERE all(x IN are_allergeni WHERE NOT x)
RETURN r.name AS ricetta, ingredienti
ORDER BY rand()
LIMIT 5 """
display(print_first_n(cq))


ricetta,ingredienti
Quick & easy party nibbles,['ingredient']
Pollo piri piri,"['Sale fino', 'Olio extravergine doliva', 'Aglio', 'Peperoncino fresco', 'Limoni', 'Paprika dolce', 'Pollo', 'Aceto di mele']"
Cheesecake golosa con fragole,"['Mascarpone', 'Burro', 'Zucchero a velo', 'Panna fresca liquida', 'Baccello di vaniglia', 'Gelatina in fogli', 'Ricotta vaccina', 'Biscotti secchi', 'Confettura di fragole']"
Culurgiones di patate viola,"['Farina 00', 'Sale fino', 'Sale fino', 'Acqua', 'Acqua', 'Semola di grano duro rimacinata', 'Olio extravergine doliva', 'Olio extravergine doliva', 'Olio extravergine doliva', 'Pepe nero', 'Menta', 'Aglio', 'Limoni', 'Pecorino sardo', 'Carciofi', 'Patate viola']"
Risotto con carote e salsiccia,"['Sale fino', 'Carote', 'Pepe nero', 'Nocciole intere spellate', 'Salsiccia', 'Brodo di carne', 'Toma', 'Riso integrale']"


### Stampa le ricette suggerite
Partendo da una ricetta ('Chocolate chia pudding'), il seguente script stampa le ricette suggerite, in base alle categorie, ai tipi di dieta e all'autore/autrice in comune.


In [93]:
ricetta = "Chocolate chia pudding"
cq = """ 
MATCH (r:Recipe {name:'"""+ ricetta +"""'})-[:COLLECTION]->(c:Collection)<-[:COLLECTION]-(r2:Recipe)
OPTIONAL MATCH (r)-[:DIET_TYPE]->(d:DietType)<-[:DIET_TYPE]-(r2)
OPTIONAL MATCH (r)<-[:WROTE]-(a:Author)-[:WROTE]->(r2)
WITH r2, COUNT(*) AS n_dati_comuni, COLLECT(DISTINCT d.name) AS diete_comuni,COLLECT(DISTINCT c.name) AS categorie_comuni, a.name AS autore_comune
RETURN r2.name AS ricetta_suggerita, categorie_comuni, diete_comuni, n_dati_comuni, autore_comune
ORDER BY n_dati_comuni DESC 
LIMIT 5
"""
display(print_first_n(cq))

ricetta_suggerita,categorie_comuni,diete_comuni,n_dati_comuni,autore_comune
"Quinoa stew with squash, prunes & pomegranate","['Easy vegan', 'Healthy vegan', 'Vegan gluten-free']","['Gluten-free', 'Vegan', 'Dairy-free', 'Vegetarian', 'Healthy']",15,null
Guacamole & mango salad with black beans,"['Healthy vegan', 'Vegan summer', 'Vegan gluten-free']","['Gluten-free', 'Healthy', 'Vegan', 'Low-calorie', 'Vegetarian']",15,null
Vegan chickpea curry jacket potatoes,"['Easy vegan', 'Healthy vegan', 'Vegan gluten-free']","['Healthy', 'Vegan', 'Vegetarian', 'Gluten-free']",12,null
Chargrilled vegetable tacos with smoky salsa,"['Easy vegan', 'Healthy vegan', 'Vegan summer']","['Vegan', 'Low-calorie', 'Healthy', 'Vegetarian']",12,null
Avocado hummus & cruditÃ©s,"['Easy vegan', 'Vegan summer']","['Vegetarian', 'Gluten-free', 'Healthy', 'Vegan', 'Low-calorie']",10,Sophie Godwin


### Quali ingredienti sono usati più spesso con la pasta? 

In [94]:
cq =""" 
MATCH (pasta:Ingredient)<-[:CONTAINS_INGREDIENT]-()-[:CONTAINS_INGREDIENT]->(i:Ingredient)
WHERE toLower(pasta.name)="pasta"
WITH i, count(*) as frequenza
RETURN i.name AS ingrediente, frequenza
ORDER BY frequenza DESC 
LIMIT 10
"""
display(print_first_n(cq))

ingrediente,frequenza
olive oil,65
parmesan,44
garlic clove,41
onion,25
lemon,23
pesto,17
parsley,14
chopped tomato,12
basil,12
tomato,11


### Ricette presenti sia su GialloZafferano, sia su BBC GoodFood

In [3]:
cq = """ 
MATCH (r_gz:Recipe {fonte:"GialloZafferano"}), (r_bbc:Recipe {fonte:"BBC GoodFood"})
WITH apoc.coll.intersection(COLLECT(DISTINCT r_gz.name), COLLECT(DISTINCT r_bbc.name)) as ricette_comuni
RETURN size(ricette_comuni) AS num_ricette_comuni, ricette_comuni
"""
ris = graph.run(cq)

In [8]:
cq = """ 
MATCH (f_gz:Fonte {name:"GialloZafferano"}), (f_bbc:Fonte {name:"BBC GoodFood"})
MATCH (r_gz:Recipe)<-[:FONTE]-(f_gz), (r_bbc:Recipe)<-[:FONTE]-(f_bbc)
WITH apoc.coll.intersection(COLLECT(DISTINCT r_gz.name), COLLECT(DISTINCT r_bbc.name)) as ricette_comuni
RETURN size(ricette_comuni) AS num_ricette_comuni, ricette_comuni
"""
ris = graph.run(cq)

# Stampo il numero di risultati
diz_ris = ris.data()[0]
print("Numero di ricette comuni trovate:", diz_ris["num_ricette_comuni"])

# Stampo le ricette su tre colonne
print("Nomi delle ricette:")
lista_ricette = diz_ris["ricette_comuni"]
for a,b,c in zip(lista_ricette[::3],lista_ricette[1::3],lista_ricette[2::3]):
    print('   {:<30}{:<30}{:<}'.format(a,b,c))
    

Numero di ricette comuni trovate: 47
Nomi delle ricette:
   Banoffee pie                  Pulled pork                   Alfajores
   Limoncello                    Pakora                        Harissa
   Hot cross buns                Colcannon                     Ceviche
   Caponata                      Pierogi                       Tarte tatin
   Acquacotta                    Coq au vin                    Fish tacos
   Ratatouille                   Cake pops                     Patatas bravas
   Key lime pie                  Quiche Lorraine               Croque madame
   Monkey bread                  Tzatziki                      Irish coffee
   Irish stew                    Raclette                      Lebkuchen
   Panettone                     Cosmopolitan                  Raita
   Tequila sunrise               Ciabatta                      Cranachan
   Baci di dama                  Bouillabaisse                 Cinnamon rolls
   Macarons                      Panforte              

## Modifiche

## Cancellazioni
